In [1]:
from pathlib import Path
#import warnings
import scanpy as sc
import scib 
import numpy as np
import pandas as pd
import sys 

import scgpt as scg
import matplotlib.pyplot as plt

plt.style.context('default')
#warnings.simplefilter('ignore', ResourceWarning)



c:\Users\annel\anaconda3\envs\scgpt_py39\lib\site-packages\scgpt\model\model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
c:\Users\annel\anaconda3\envs\scgpt_py39\lib\site-packages\scgpt\model\multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
c:\Users\annel\anaconda3\envs\scgpt_py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("\nAll attributes in scg.tasks:")
for attr in dir(scg.tasks):
    if not attr.startswith('_'):
        print(f"- {attr}")


All attributes in scg.tasks:
- GeneEmbedding
- cell_emb
- embed_data
- embed_data_cpu
- get_batch_cell_embeddings
- grn


In [3]:
# Check if HVG AnnData file exists in save directory
hvg_file = Path("C:/Users/annel/OneDrive/Documenten/Machine Learning/scGPT/save/adata_hvg.h5ad")

if hvg_file.exists():
    print(f"Loading existing HVG AnnData file from {hvg_file}")
    adata = sc.read_h5ad(hvg_file)
    print(f"Loaded AnnData with shape: {adata.shape}")
else:
    print("No existing HVG AnnData file found in save directory")

# Define repo dir 
import sys

repo_dir = Path.cwd().parent.absolute()
sys.path.append(str(repo_dir))

from src.utils import setup_data_dir
setup_data_dir()
data_dir = repo_dir / "data"

# Set up of the model directory 
model_dir = repo_dir / "scGPT_data" / "scGPT_CP"

print(f"Repository directory: {repo_dir}")
print(f"Model directory: {model_dir}")

gene_col = "gene_name"
cell_type_key = "celltype" #where the cell names are stored
batch_key = "tech" #show from different methods 
N_HVG = 1800 #number of high variable genes 





Loading existing HVG AnnData file from C:\Users\annel\OneDrive\Documenten\Machine Learning\scGPT\save\adata_hvg.h5ad
Loaded AnnData with shape: (16382, 1800)
Repository directory: c:\Users\annel\OneDrive\Documenten\Machine Learning\scGPT
Model directory: c:\Users\annel\OneDrive\Documenten\Machine Learning\scGPT\scGPT_data\scGPT_CP


In [8]:
Embed_File_Path = Path(r"C:\Users\annel\OneDrive\Documenten\Machine Learning\scGPT\data\human_pancreas_norm_complexBatch_embedded.h5ad")

adata_embed = sc.read_h5ad(Embed_File_Path)

print(adata_embed)
# Create a dataframe from the scGPT embeddings
embeddings_df = pd.DataFrame(adata_embed.obsm["X_scGPT"])

# Display the first few rows
print("\nFirst few rows of scGPT embeddings:")
print(embeddings_df.head())



AnnData object with n_obs × n_vars = 16382 × 1722
    obs: 'tech', 'celltype', 'size_factors'
    var: 'gene_name', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'id_in_vocab'
    uns: 'hvg', 'log1p'
    obsm: 'X_scGPT'
    layers: 'counts'

First few rows of scGPT embeddings:
        0         1         2         3         4         5         6    \
0  0.073707  0.003733  0.012353  0.045787  0.028686 -0.056230 -0.011507   
1  0.071376 -0.001319  0.009428  0.041126  0.030103 -0.045669 -0.007892   
2  0.071767 -0.001087  0.003495  0.024505  0.013971 -0.056813 -0.007876   
3  0.068650 -0.009042  0.010772  0.032528  0.003143 -0.059393 -0.001724   
4  0.050278 -0.014192  0.012542  0.025254  0.020374 -0.052632  0.018501   

        7         8         9    ...       502       503       504       505  \
0 -0.010013  0.009184 -0.000568  ...  0.008169  0.018345 -0.009001 -0.054426   
1 -0.011339  0.009670 -0.014000  ... -0.006212  0.013708 -0.002215 -0.0258

In [ ]:
# Randomly split the data into reference and query sets (80% reference, 20% query)
np.random.seed(42)  # For reproducibility
mask = np.random.rand(len(adata_embed)) < 0.8

# Create a column indicating whether each cell is from reference or query set
adata_embed.obs["is_ref"] = ["Reference" if m else "Query" for m in mask]

adata_concat.obs["is_ref"] = adata_concat.obs["is_ref"].astype("category")

# Create reference and query datasets
adata_ref = adata_embed[mask].copy()
adata_query = adata_embed[~mask].copy()

print(f"Reference dataset shape: {adata_ref.shape}")
print(f"Query dataset shape: {adata_query.shape}")

adata_query.obs["cell_type"] = adata_query.obs["celltype"]


Reference dataset shape: (13110, 1722)
Query dataset shape: (3272, 1722)


In [20]:
ref_embeddings = adata_ref.obsm["X_scGPT"]
query_embeddings = adata_query.obsm["X_scGPT"]



